# convert coco format to yolov5 format

In [ ]:
!fiftyone convert \
    --input-dir cabbages_dataset \
    --input-type fiftyone.types.COCODetectionDataset \
    --output-dir cabbages_YOLO \
    --output-type fiftyone.types.YOLOv5Dataset

# visualize a image and a bounding box

In [ ]:
import fiftyone as fo

name = "my-dataset"
dataset_dir = "cabbages_YOLO"

# The splits to load
# splits = ["train", "val"]
splits = ["val"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    split="val",
    label_field="ground_truth",
)
# View summary info about the dataset
print(dataset)

In [ ]:
sample = dataset.first()
print(sample.ground_truth.detections[0])

In [ ]:
print(dataset.count())
print(dataset.count("ground_truth.detections"))

In [ ]:
session = fo.launch_app(dataset)

# convert YOLOv5 format to pascal format

In [ ]:
import os
import shutil
import torch
import torchvision
from torchvision.io.image import read_image
from torchvision.transforms.v2 import functional as F

In [ ]:
data_root = 'data/fold1'

os.listdir(data_root)
os.makedirs(f'{data_root}_pascal', exist_ok=True)
for dataset in ['train', 'val', 'test']:
    os.makedirs(f'{data_root}_pascal/{dataset}', exist_ok=True)
    os.makedirs(f'{data_root}_pascal/{dataset}/images', exist_ok=True)
    os.makedirs(f'{data_root}_pascal/{dataset}/labels', exist_ok=True)
    
    for folder in ['images', 'labels']:
        for name in os.listdir(f'{data_root}/{dataset}/{folder}'):
            if folder == 'images':
                    src = f'{data_root}/{dataset}/{folder}/{name}'
                    dst = f'{data_root}_pascal/{dataset}/{folder}/{name}'
                    shutil.copyfile(src, dst)
                    
            else:
                # Load image
                image_name = name.replace('.txt', '.png')
                image_path = f'{data_root}/{dataset}/images/{image_name}'
                image = read_image(image_path)
                if image is None:
                    raise Exception(f"Error reading image: {image_path}")
                heigth, width = F.get_size(image)
                
                # load label
                label_path = f'{data_root}/{dataset}/{folder}/{name}'
                with open(label_path, "r") as file:
                    lines = file.read().split('\n')
        #         print(lines)

                # convert YOLOv8 format to xyxy format
                boxes = []
                labels = []
                for i in range(len(lines)):
                    line = lines[i].split()
                    # denormalize
                    boxes.append([
#                         int(round(float(line[1])*width)),
#                         int(round(float(line[2])*heigth)),
#                         int(round(float(line[3])*width)),
#                         int(round(float(line[4])*heigth))
                        
                        float(line[1])*width,
                        float(line[2])*heigth,
                        float(line[3])*width,
                        float(line[4])*heigth                        
                    ])
                    labels.append(int(line[0]))      # non shift
    #                 labels.append(int(line[0])+1)     # shift 1 class 

#                 print(boxes)
                # convert format
                boxes = torchvision.ops.box_convert(
                    boxes=torch.tensor(boxes, dtype=torch.float32),
                    in_fmt='cxcywh',
                    out_fmt='xyxy'
                )
#                 print(boxes)
                
                lines = []
                for i in range(len(boxes)):
                    line = f'{labels[i]} {round(boxes[i][0].item())} {round(boxes[i][1].item())} {round(boxes[i][2].item())} {round(boxes[i][3].item())}'
                    lines.append(line)
                    
                with open(f'{data_root}_pascal/{dataset}/{folder}/{name}', "w") as file:
                    file.write('\n'.join(lines))


            print(dataset, folder, name)